## 📝 安裝套件

In [83]:
!pip install jieba -U
!pip install httpx -U
!pip install sklearn -U
!pip install ckiptagger[tf,gdown] -U
!pip install BeautifulSoup4 -U
'''
1. 抓資料
2. 資料清洗
3. 長詞優先斷詞+jieba斷詞+ckiptagger斷詞
4. tf-idf取出關鍵字
5. 計算文件相似度
'''

'\n1. 抓資料\n2. 資料清洗\n3. 長詞優先斷詞+jieba斷詞+ckiptagger斷詞\n4. tf-idf取出關鍵字\n5. 計算文件相似度\n'

## 📝 針對標點符號斷行

In [42]:
import re
def splitSentense(text, delimiter):
    return re.split(delimiter, text)
    
delimiter = "，|。|、|（|）|／|《|》|】|【|「|」|；|："
allNews = ['虛擬貨幣「比特幣期貨」要來了！美國監管機構美國商品期貨交易委員會（CFTC）一日宣布將放行比特幣期貨，允許芝加哥商品交易所（CME）和芝加哥選擇權交易所（CBOE）推出相關期貨合約，因為兩交易所已證明擬推出的合約和交易安全符合必要的監管規定；這成為推動主流投資人買賣此價格高度波動的數位貨幣的重大一步。']
sentenceAry = []
for rec in allNews:
    text = rec
    sentenceAry += splitSentense(text,delimiter)
sentenceAry    

['虛擬貨幣',
 '比特幣期貨',
 '要來了！美國監管機構美國商品期貨交易委員會',
 'CFTC',
 '一日宣布將放行比特幣期貨',
 '允許芝加哥商品交易所',
 'CME',
 '和芝加哥選擇權交易所',
 'CBOE',
 '推出相關期貨合約',
 '因為兩交易所已證明擬推出的合約和交易安全符合必要的監管規定',
 '這成為推動主流投資人買賣此價格高度波動的數位貨幣的重大一步',
 '']

## 📝 ngram + 長詞優先斷詞 => 取出特殊關鍵詞

In [48]:
import operator
def removeKey(text, keyword):
    textAry= text
    for key in keyword:
        textAry = ''.join(textAry.split(key))
    return textAry
def ngram(input_sentence, n = 2):
    word_dic = {}
    sentence  = input_sentence
    for i in range(0, len(sentence) - n + 1):        
        if sentence[i:i+n] not in word_dic:
            word_dic[sentence[i:i+n]] = 1
        else:
            word_dic[sentence[i:i+n]] = word_dic[sentence[i:i+n]] + 1
    return word_dic    
keywords=['期貨','核准']        
ret_terms={}
words_freq    = []
for term_length in range(4,1,-1):
    word_dic = {}
    for sentence in sentenceAry:
        text_list = removeKey(sentence,keywords)  
        ngram_words = ngram(text_list,term_length) 
        for word in ngram_words:
            if word not in word_dic:
                word_dic[word] = 1
            else:
                word_dic[word] += ngram_words[word]   
    for word in word_dic:
        if word_dic[word] >= 5:
            keywords.append(word)            
            ret_terms.update({word:word_dic[word]})
sorted_terms = sorted(ret_terms.items(),key=operator.itemgetter(1),reverse=True) 
sorted_terms


[('交易', 5)]

## 📝 tf-idf

In [49]:
import jieba.analyse
sentence = '''有外貌美麗如女星范冰冰的馮姓女子2008年與陳姓富二代訂婚後無故悔婚，但馮女不告而別2年後，忽然回頭請陳男金援，癡情的陳男不但不計前嫌，對馮女數十次索討生日、情人節等禮物及資助學費等一概答應，竟連她結婚生子後要求生活費，也大方匯給10萬元，直到陳男也結婚，妻發現他竟贈百萬名畫給馮女，陳男才在妻逼迫下告馮女詐欺、侵占，並請求返還借款533萬餘元，但檢方認定陳男自願贈與，將馮女不起訴，台北地院也僅認定2013年間馮女謊稱買房討「房貸頭期款」部分陳男求償有理，判馮女須賠60萬元。'''
tags = jieba.analyse.extract_tags(sentence,3, withWeight=True)
tags


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\smart\AppData\Local\Temp\jieba.cache
Loading model cost 0.557 seconds.
Prefix dict has been built successfully.


[('馮女', 0.6210268832675324),
 ('萬元', 0.3105134416337662),
 ('認定', 0.3105134416337662)]

## 📝 查詢相似文件

In [82]:
import pandas
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

news = pandas.read_excel('news.xlsx')

corpus = []
titles = []
for article in news.iterrows():
    titles.append(article[1].title)
    corpus.append(' '.join(jieba.cut(article[1].content)))
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(corpus)
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
# weight = tfidf.toarray()

def get_similarity_sentence(articleid):
    print('[查詢文章]:{}'.format(titles[articleid]))
    cosine_similarities = cosine_similarity(tfidf[articleid], tfidf).flatten()
    related_docs_indices = cosine_similarities.argsort()[-2::-1]
    for idx in related_docs_indices:
        if cosine_similarities[idx] > 0.09:
            print('[相關文章]:{} {}'.format(titles[idx], cosine_similarities[idx]))
get_similarity_sentence(15)          

[查詢文章]:【不斷更新】槍手闖美賭城音樂會　CNN：多人中彈送醫
[相關文章]:【不斷更新】彭博：就在周五　加泰隆尼亞可能宣布獨立 0.09034792117666549


In [122]:
import httpx
from bs4 import BeautifulSoup
import time,asyncio

headers = {'User-Agent': 'GoogleBot'}
start=1
end=30
root_url='https://www.ithome.com.tw'
url=f'{root_url}/devops'

async def get_news(news_url,page):
    req=httpx.get(f'{news_url}?page={page}',headers=headers)
    res=req.text
    root=BeautifulSoup(res,'lxml')
    news_list=root.find_all('span',attrs={'class':'views-field views-field-created'})
    for news in news_list:
        news_div=news.span.div
        categories=list(map(lambda x:x.text,news_div.find('p',attrs={'class':'category'}).find_all('a')))
        print(categories)
        title_div=news_div.find('p',attrs={'class':'title'})
        title=title_div.a.text.strip()
        href=f"{root_url}{title_div.a['href']}"
        print(title)
        print(href)
        summary=news_div.find('div',attrs={'class':'summary'}).text.strip()
        print(summary)
        post_date=news_div.find('p',attrs={'class':'post-at'}).text.strip()
        print(post_date)
        print(f'--------------{page}-----------')
        
async def main():
    task_list=[]
    for i in range(start,end+1):
        task_list.append(get_news(url,i))
        print(i)
    await asyncio.gather(*task_list)

s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")        
   


1
2
3
4
5
6
7
8
9
10
['新聞', 'Line臺灣SRE團隊', 'Line臺灣技術長', '陳鴻嘉', '網站可靠性工程', 'google', 'SRE', 'Site Reliability Engineering', 'Line臺灣', 'K8s', 'Kubernetes', '服務觀測']
【臺灣SRE實例：Line臺灣】如何確保Line服務天天不中斷，專責SRE扮演開發與維運的橋樑
https://www.ithome.com.tw/news/144121
去年，Line臺灣工程團隊成立了一支專責的SRE維運團隊，就是要為各服務的專案團隊找到開發和維運的最佳作法，進一步確保服務可靠性
2021-05-06
--------------1-----------
['新聞', '容器週報', 'K8s', 'Docker', 'OpenShift', '容器資安', 'DevOps', 'Kubernetes', 'IT周報']
Container周報第137期：K8s今年第一個新版有50項更新，紅帽揭露OpenShift零信任架構未來3大發展方向
https://www.ithome.com.tw/news/143911
OpenShift在三月公布的發展藍圖中，接露了資安上的未來發展方向，將內建零信任架構，包括三大方向，部建階段的偵測能力、部署階段的保護機制和執行階段的應變機制
2021-04-19
--------------1-----------
['新聞', 'RPA', 'myInvenio', 'IBM', '收購', '企業流程驅動', '流程採礦', 'Cloud']
搶進RPA市場，IBM收購流程採礦廠商myInvenio
https://www.ithome.com.tw/news/143877
IBM要將myInvenio的流程自動化、文件處理、流程採礦、RPA等技術，納入自家雲端流程服務，以切入日益競爭的RPA市場
2021-04-16
--------------1-----------
['新聞', 'ML Pipeline', 'AI開發', 'MLOps', 'AI落地', '協作', 'DevOps', '人工智慧', 'AI應用', 'MLOps解決  方案', 'ML模型自動化', 'AI維運'

In [124]:
import httpx
from bs4 import BeautifulSoup
import time,asyncio
headers = {'User-Agent': 'GoogleBot'}
start=1
end=30
root_url='https://www.ithome.com.tw'
url=f'{root_url}/devops'
def get_news(news_url,page):
    req=httpx.get(f'{news_url}?page={page}',headers=headers)
    res=req.text
    root=BeautifulSoup(res,'lxml')
    news_list=root.find_all('span',attrs={'class':'views-field views-field-created'})
    for news in news_list:
        news_div=news.span.div
        categories=list(map(lambda x:x.text,news_div.find('p',attrs={'class':'category'}).find_all('a')))
        #print(categories)
        title_div=news_div.find('p',attrs={'class':'title'})
        title=title_div.a.text.strip()
        href=f"{root_url}{title_div.a['href']}"
        #print(title)
        #print(href)
        summary=news_div.find('div',attrs={'class':'summary'}).text.strip()
        #print(summary)
        post_date=news_div.find('p',attrs={'class':'post-at'}).text.strip()
        #print(post_date)
        #print(f'--------------{page}-----------')
def main():
    for i in range(start,end+1):
        get_news(url,i)

s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")        
   


['新聞', 'Line臺灣SRE團隊', 'Line臺灣技術長', '陳鴻嘉', '網站可靠性工程', 'google', 'SRE', 'Site Reliability Engineering', 'Line臺灣', 'K8s', 'Kubernetes', '服務觀測']
【臺灣SRE實例：Line臺灣】如何確保Line服務天天不中斷，專責SRE扮演開發與維運的橋樑
https://www.ithome.com.tw/news/144121
去年，Line臺灣工程團隊成立了一支專責的SRE維運團隊，就是要為各服務的專案團隊找到開發和維運的最佳作法，進一步確保服務可靠性
2021-05-06
--------------1-----------
['新聞', '容器週報', 'K8s', 'Docker', 'OpenShift', '容器資安', 'DevOps', 'Kubernetes', 'IT周報']
Container周報第137期：K8s今年第一個新版有50項更新，紅帽揭露OpenShift零信任架構未來3大發展方向
https://www.ithome.com.tw/news/143911
OpenShift在三月公布的發展藍圖中，接露了資安上的未來發展方向，將內建零信任架構，包括三大方向，部建階段的偵測能力、部署階段的保護機制和執行階段的應變機制
2021-04-19
--------------1-----------
['新聞', 'RPA', 'myInvenio', 'IBM', '收購', '企業流程驅動', '流程採礦', 'Cloud']
搶進RPA市場，IBM收購流程採礦廠商myInvenio
https://www.ithome.com.tw/news/143877
IBM要將myInvenio的流程自動化、文件處理、流程採礦、RPA等技術，納入自家雲端流程服務，以切入日益競爭的RPA市場
2021-04-16
--------------1-----------
['新聞', 'ML Pipeline', 'AI開發', 'MLOps', 'AI落地', '協作', 'DevOps', '人工智慧', 'AI應用', 'MLOps解決  方案', 'ML模型自動化', 'AI維運', 'Line']
Line AI開發的關